<div class="alert alert-block alert-success">
    <h1 align="center">Missing Values (Imputation) in a Nutshell</h1>
    <h3 align="center">PreProcessing before Training model</h3>
    <h4 align="center"><a href="http://www.iran-machinelearning.ir">Soheil Tehranipour</a></h5>
</div>

# Introduction

Missing values is one of the most common problems that you will come across when performing data analysis. Furthermore, machine learning models require that a dataset does not contain any missing values before they can be fitted to the data. Therefore, it is crucial that we learn how to properly handle them.

There are mainly two ways that we can deal with missing values in a dataset:

1. If there are only a few rows with missing values or if a column has an overwhelming number of missing values, we can simply drop them from the dataset without running into the risk of losing too much information. 
2. An alternative approach to handling missing values is via imputation. Imputation is the process whereby the missing values in a dataset are replaced with some substituted values. 

For the purpose of this tutorial, we will examine the process of imputation and more specifically, we will learn the difference between a univariate approach to imputing versus a multivariate approach to imputing. 

Univariate imputation implies that we are only considering the values of a single column when performing imputation. Multivariate imputation, on the other hand, involves taking into account other features in the dataset when performing imputation.

The multivariate approach to imputing is generally preferred over the univariate approach as it is more robust and provides our model with a more accurate representation of the missing values in order to make better predictions.

In this tutorial, we will explore 3 different imputation techniques with reference to the [Titanic dataset](https://www.kaggle.com/c/titanic/data) on Kaggle:

- [Simple imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)
- [Iterative imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer)
- [KNN Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html)

# 1. Import libraries

In [ ]:
# Data wrangling
import pandas as pd
import numpy as np 

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Machine learning
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# 2. Prepare dataset

# 2.1 Import and read data

In [ ]:
data = pd.read_csv(r"C:\Users\soso\Desktop\Maktbakhoone\Scikit-Learn Tips\00 -  Missing Value/train.csv")
data.head()

# 2.2 Drop unwanted columns 

For the purpose of this tutorial, I proceeded to drop the PassengerId, Name, Ticket and Cabin columns from the original dataset. 

In [ ]:
# Drop PassengerId, Name, Ticket and Cabin columns 

data = data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
data.head()

# 2.3 Drop rows with missing Embarked values

In [ ]:
# Missing data

missing = data.isnull().sum()
missing = missing[missing > 0].sort_values(ascending = False)
missing

Because there are only two rows with missing Embarked values, I decided to simply drop those two rows from the dataset, leaving us with 177 rows with missing Age values. 

In [ ]:
# Drop rows with missing Embarked values 

print("Before: ", data.shape)
data = data.loc[data['Embarked'].notnull(), :]
print("After: ", data.shape)

In [ ]:
# Check missing values again

missing = data.isnull().sum()
missing[missing > 0]

# 3. Explore Age feature

Before we perform imputation on the Age column, let's briefly explore the Age feature. 

In [ ]:
# Plot heatmap 

correlation = data.corr()
plt.figure(figsize = (7, 5))
sns.heatmap(correlation, annot = True, fmt = '.2f', cmap = 'coolwarm')
plt.title('Correlation between features')

In [ ]:
# Age correlation with other features 

correlation['Age'].sort_values(ascending = False)[1:]

As we can see, the Age feature is slightly positively correlated with the Fare feature. In other words, passengers that are older generally pay a higher fare. 

In [ ]:
# Plot passenger age distribution

plt.figure(figsize = (8, 5))
sns.distplot(data['Age'], label = 'Skewness: {:.2f}'.format(data['Age'].skew()))
plt.legend(loc = 'best')
plt.title('Passenger Age Distribution')

In [ ]:
# Age summary statistics 

data['Age'].describe()

# 4. Missing values imputation

In this section, we will look at 3 different imputation techniques:

1. Simple imputer
2. Iterative imputer
3. KNN Imputer

Here, I have created a sample dataframe to demonstrate the differences between the 3 techniques. 

In [ ]:
# Create sample dataframe

df = pd.DataFrame({'SibSp': [1, 1, 0, 1, 0, 0], 
                   'Fare': [7.25, 71.2833, 7.925, 53.1, 8.05, 8.4583], 
                   'Age': [22, 38, 26, 35, 35, np.nan]})
df

# 4.1 Simple imputer

Simple imputer follows a univariate approach to imputing missing values i.e. it only takes a single feature into consideration. Some of the most common uses of simple imputer are:

- Mean
- Median
- Most frequent (mode)

In [ ]:
print("Average age: ", df['Age'].mean())
simple_imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')
simple_imp.fit_transform(df)

As we can see, simple imputer has filled the missing value in the Age column with the average age which is 31.2.

# 4.2 Iterative imputer

Iterative imputer is an example of a multivariate approach to imputation. It models the missing values in a column by using information from the other columns in a dataset. More specifically, it treats the column with missing values as a target variable while the remaining columns are used are predictor variables to predict the target variable. 

In our sample data frame, the Age column has one missing value on row 6 and is therefore assigned as the target variable in this scenario. This leaves the SibSp and Fare columns as our predictor variables. 

Iterative imputer will use the first 5 rows of the data frame to train a predictive model. Once the model is ready, it will then values in the SibSp and Fare columns of row 6 as inputs and predict the Age value for that row.

#### It is a data transform that is first configured based on the method used to estimate the missing values. By default, a BayesianRidge model is employed that uses a function of all other input features. Features are filled in ascending order, from those with the fewest missing values to those with the most.



In [ ]:
iterative_imp = IterativeImputer()
iterative_imp.fit_transform(df)

As we can see, the value predicted under iterative imputer is different to that under simple imputer.

This is a more accurate approach to predict the missing Age value as it takes other features in our dataframe into account. 

# 4.3 KNN imputer

Last but not least, we have KNN Imputer which is another multivariate imputation technique. KNN Imputer scans our dataframe for k nearest observations to the row with missing value. It will then proceed to fill the missing value with the average of those nearest observations. 

Here, I have set k to equal to 2 or in other words, I want KNN imputer to look for 2 observations that are nearest to row 6 and fill the missing age with the average age of those 2 rows.

In [ ]:
knn_imp = KNNImputer(n_neighbors = 2)
knn_imp.fit_transform(df)

As a result, KNN imputer has taken row 3 and row 5 as the nearest observations for row 6.

Therefore, the average age between row 3 and row 5 is (26 + 35) / 2 = 30.5. 

# 5. Model accuracy under simple imputer and iterative imputer

Now that we have a better understanding of how the different imputers work, we can move on to apply these techniques to our Titanic dataset and compare the model accuracy under each approach.

We should expect to see our model perform better under multivariate imputation than univariate imputation as multivariate imputation provides a more accurate prediction of the missing values and thus allowing our model to make better predictions. 

In this section, we will build a column transformer which consists of a OneHotEncoder for encoding the Sex and Embarked columns as well as an imputer to impute the missing values in the Age column.

Following that, we will chain the column transformer with a random forest classifier to predict the surival of the passengers on the Titanic. Finally, we will perform 10-fold cross-validation to compare the prediction results under univariate imputation versus under multivariate imputation. 

In [ ]:
data.head()

In [ ]:
# Get predictor and target variables 

X = data.drop('Survived', axis = 1)
Y = data['Survived']

In [ ]:
# Predictor and target variables shape

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)

# 5.1 Univariate imputation (simple imputer)

In [ ]:
# Instantiate OneHotEncoder for Sex and Embarked columns
ohe = OneHotEncoder()

# Instantiate simple imputer 
simple_imp = SimpleImputer(missing_values = np.nan, strategy = 'mean')

# Instantiate model
rf = RandomForestClassifier()

# Make column transformer with simple imputer
column_transform = make_column_transformer(
    (ohe, ['Sex', 'Embarked']),
    (simple_imp, ['Age']),
    remainder = 'passthrough')

# Pipeline
pipe = make_pipeline(column_transform, rf)

# 10-fold cross-validation
cross_val_score(pipe, X, Y, cv = 10, scoring = 'accuracy').mean()

# 5.2 Multivariate imputation (iterative imputer)

In [ ]:
# Instantiate OneHotEncoder for Sex and Embarked columns
ohe = OneHotEncoder()

# Instantiate iterative imputer
iterative_imp = IterativeImputer()

# Instantiate model
rf = RandomForestClassifier()

# Make column transformer with iterative imputer
column_transform = make_column_transformer(
    (ohe, ['Sex', 'Embarked']),
    (iterative_imp, ['Age']),
    remainder = 'passthrough')

# Pipeline
pipe = make_pipeline(column_transform, rf)

# 10-fold cross-validation 
cross_val_score(pipe, X, Y, cv = 10, scoring = 'accuracy').mean()

Although not a drastic difference, we can conclude that our model performed better under multivariate imputation as it achieved a higher mean cross-validation score.

# 6. Conclusion

In this tutorial, we have learned the difference between univariate imputation and multivariate imputation. In addition, We also looked at 3 different imputation techniques within Scikit-learn which include simple imputer, iterative imputer and KNN imputer. 

After comparing the prediction accuracy of our model using simple imputer versus iterative imputer, we can conclude that multivariate imputation results in better model predictions due to its lower mean cross-validation score.

# 7. References

- [Scikit-learn imputation of missing values](https://scikit-learn.org/stable/modules/impute.html)
- [Impute missing values using KNNImputer and IterativeImputer](https://www.youtube.com/watch?v=m_qKhnaYZlc)

<img src="https://webna.ir/wp-content/uploads/2018/08/%D9%85%DA%A9%D8%AA%D8%A8-%D8%AE%D9%88%D9%86%D9%87.png" width=50% />